<a href="https://colab.research.google.com/github/vivek11416/tensorflow/blob/master/TF_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
#MNIST dataset parameters
num_classes = 10 # 0 to 9 digits
num_features = 28*28

# training parameters
learning_rate = 0.01
training_steps = 1000
batch_size = 256
displaystep = 50

In [4]:
#prepare MNIST dataset
from tensorflow.keras.datasets import mnist
(x_train,y_train),(x_test,y_test) = mnist.load_data()

# convert to float32
x_train,x_test = np.array(x_train,np.float32),np.array(x_test,np.float32)
#flatten imaged to 1-d vector
x_train,x_test = x_train.reshape([-1,num_features]),x_test.reshape([-1,num_features])
#normalize image
x_train,x_test = x_train / 255. , x_test / 255.

11501568/11490434 [==============================] - 0s 0us/step


In [5]:
# shuffle and batch data
train_data = tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [6]:
#Weight and Bias
W = tf.Variable(tf.ones([num_features,num_classes]),name = "weight")
b = tf.Variable(tf.zeros([num_classes]),name="bias")

# Logistic regression (Wx+b)
def logistic_regression(x):
  #softmax to normalize the logits to a probability distribution
  return tf.nn.softmax(tf.matmul(x,W) + b)

#cross-Entroy loss funtion
def cross_entropy(y_pred,y_true):
  y_true = tf.one_hot(y_true,depth=num_classes)
  #clip prediction value to avoid log(0) error
  y_pred = tf.clip_by_value(y_pred,1e-9,1.)
  # compute cross-entropy
  #return tf.reduce_mean(-tf.reduce_sum(y_true*tf.math.log(y_pred)),1) #https://stackoverflow.com/questions/42521400/calculating-cross-entropy-in-tensorflow
  return tf.nn.softmax_cross_entropy_with_logits(y_pred,y_true)

#Accuracy Metric
def accuracy(y_pred,y_true):
  correct_prediction = tf.equal(tf.argmax(y_pred,1),tf.cast(y_true,tf.int64))
  return tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

#stochastic gradient descent optimizer
optimizer = tf.optimizers.SGD(learning_rate)